In [25]:
#### tuneBymwBen(wlist=[], path='', wfile='')
# tuneReport(wlist=[], path='', wfile='')
# txt2wl(path='', wfile='')
# hw4learn(wdict={}, wlist=[], path='', wfile='', excluf='E:\\1Now\\taglist\\2939exclu.txt')
# compareWordlist(l1=[], l2=[], path='', f1='', f2='')


# tuneBymwBen(path='E:\\1Now\\taglist\\', wfile='testmwBen.txt') 
# tuneReport(path='E:\\1Now\\taglist\\', wfile='testmwBen.txt')

### A typcial usage example: create words to learn Anki search string from an article
# wlraw = txt2wl('E:\\1Now\\mae\\article\\', 'pd1_18.txt')
# rnt = tuneReport(wlist=wlraw)
# # rnt = tuneReport(path='E:\\1Now\\taglist\\', wfile='46sounds2.txt')
# sound4learn = hw4learn(wdict=rnt)


# compareWordlist(path="E:\\1Now\\taglist\\", f1="84en_fr.txt", f2="mwlBenchmark.txt")

In [19]:

import codecs
import re
import string

def tuneBymwBen(wlist=[], path='', wfile=''):        
    
    if wfile:
        print '\n\n------', path+wfile, '------'
        with codecs.open(path+wfile, 'r', 'utf-8') as f:
            txtraw = f.read().splitlines()
        txtraw = [x.strip() for x in txtraw]
    elif wlist:
        txtraw = [x.strip() for x in wlist]
    else:
        print 'function argumenents error'
        return -1
    
    print '* has words : ', len(txtraw)
    print '* has unique word number: ', len(set(txtraw))
    dupinraw = set([x for x in txtraw if txtraw.count(x) > 1])
    if dupinraw: print '* has duplicates: ', dupinraw
   
        
    with codecs.open("E:\\1Now\\taglist\\mwlBenchmark.txt", 'r', 'utf-8') as f:    
        lines = f.read().splitlines()

    # generate mwBen dictionary    
    hws = [] # sorted unique hws
    hwsC = [] # sorted unique capitalized hws
#     mwBens = []
    mwBenDerivs = []
    mwBenInfls = []
    
    
    for line in lines:
#         if len(line) < 1:
#             print 'len(line)<1', line 
#             return -1
        hw = line.split(';')[2]
        wtype = line.split(';')[0]
                
        entry = {}
        if wtype == '2':            
            entry['hw'] = hw
            entry['deriv'] = line.split(';')[1] # derivative
            mwBenDerivs.append(entry)
        elif wtype == '3':            
            entry['hw'] = hw
            entry['infl'] = line.split(';')[1] # inflection
            mwBenInfls.append(entry)
                
        hws.append(hw)
        if hw[0] in string.lowercase[0:26]:
            hwsC.append(hw.capitalize())
        
    hws = sorted(set(hws))
    hwsC = sorted(set(hwsC))        


    # begin to check the word list against mw benchmark dictionary
    #  Word; Its-hw (Head Word)
    #
    # Three cases
    # 1. word is exactly same as hw or is a capitalized hw (like 'Walk') or word.capitalize() is in hw (like 'leo')
    # 2. word is a derivative or is a capitalized derivative (like 'Walks') or word.capitalize() is in derivate (like 'leos')
    # 3. word is not in mwBen

    
    hwreturn = [] # hws returned by this function
    
    wordhw = [] # word is exactly same as hw
    wordhwL = [] # word.lower() is a hw, like 'Walk'
    wordhwC = [] # word.capitalize() is a hw, like 'leo'
    
    wordDeriv = [] # raw derivatives or capitalized derivatives or derivative.lower()
    fineDeriv = [] # fine derivateivs are in mwBen
    hwofDeriv = [] # hws of derivatives
    
    wordInfl = [] # raw inflections or capitalized inflections or inflection.lower()
    fineInfl = [] # fine inflections are in mwBen
    hwofInfl = [] # hws of inflections
    
    wordnotBen = [] # word is not in mwBen

    for word in txtraw:
        hwflag = 0 # suppose that word is not a hw
        derivflag = 0 # suppose that word is not a derivative
        inflflag = 0 # suppose that word is not a inflection
        
        # first check if word is a hw
        if word in hws:
            wordhw.append(word)
            hwreturn.append(word)
            hwflag = 1
        elif word.capitalize() in hws: # like leo
            wordhwC.append(word)
            hwreturn.append(word.capitalize())
            hwflag = 1            
        elif word.capitalize() in hwsC: # like Walk, tAke
            wordhwL.append(word)
            hwreturn.append(word.lower())
            hwflag = 1

        # if word isn't a hw, then check if it's a derivative
        if hwflag == 0:
            dercnt = 0
            for mwben in mwBenDerivs:                
                if word == mwben['deriv'] or word == mwben['deriv'].capitalize() or word.capitalize() == mwben['deriv']:
                    wordDeriv.append(word)
                    fineDeriv.append(mwben['deriv'])
                    hwofDeriv.append(mwben['hw'])
                    derivflag = 1
                    print '---derivative: ', word, '--- hw: ', mwben['hw']
                    dercnt += 1
            if dercnt > 1: print word, '--- derivate has multiple hws --- ', dercnt

        if hwflag == 0 and derivflag == 0:
            inflcnt = 0
            for mwben in mwBenInfls:                
                if word == mwben['infl'] or word == mwben['infl'].capitalize() or word.capitalize() == mwben['infl']:
                    wordInfl.append(word)
                    fineInfl.append(mwben['infl'])
                    hwofInfl.append(mwben['hw'])
                    inflflag = 1
                    print 'inflection: ', word, '--- hw: ', mwben['hw']
                    inflcnt += 1
            if inflcnt > 1: print word, '--- inflection has multiple hws --- ', inflcnt                
                
                
        if hwflag == 0 and derivflag == 0 and inflflag == 0:
            if word.find(' ') == -1:
                wordnotBen.append(word)
                print '**************^^^^^^^!!@@@@@*************** is not in mwBen: ', word
            else:
                print '>>> >>> >>> >>> >> not a single word: ', word
                for x in word.split(' '):
                    if x in hws: hwreturn.append(x)
                    elif x.capitalize() in hws: hwreturn.append(x.capitalize())
                    elif x in hwsC: hwreturn.append(x.lower())
                    else:
                        wdflagx = 0 
                        for mwben in mwBenDerivs:                
                            if x == mwben['deriv'] or x == mwben['deriv'].capitalize() \
                            or x.capitalize() == mwben['deriv']:
                                wordDeriv.append(word)
                                fineDeriv.append(mwben['deriv'])
                                hwofDeriv.append(mwben['hw'])
                                wdflagx = 1
                        if wdflagx == 0:
                            for mwben in mwBenInfls:                
                                if x == mwben['infl'] or x == mwben['infl'].capitalize() \
                                or x.capitalize() == mwben['infl']:
                                    wordInfl.append(word)
                                    fineInfl.append(mwben['infl'])
                                    hwofInfl.append(mwben['hw'])
                                    wdflagx = 1
                        if wdflagx == 0:
                            wordnotBen.append(x)
                            print '**************^^^^^^^!!@@@@@*************** really is not in mwBen: ', x
                                
                            
            
    
    if wordhw: print '* same as hw: ', len(wordhw)  #,' -- ', wordhw
    if wordhwL: print '* capitalized hw: ', len(wordhwL)  #,' -- ', wordhwL
    if wordhwC: print '* hw typo, should be capitalized: ', len(wordhwC)  #,' -- ', wordhwC
    
    if wordDeriv: print '* words are derivatives: ', len(wordDeriv) #,' -- ', wordDeriv
    if wordInfl: print '* words are inflections: ', len(wordInfl) #,' -- ', wordInfl

    if wordnotBen:
        print '\n**************##########***************** All not in mwBen: ', len(wordnotBen), '\n', wordnotBen
        
#     if len(txtraw) == len(wordhw) + len(wordhwL) + len(wordhwC) + len(wordDeriv) + len(wordnotBen):         
#     else: print "--- words' numbers don't mattch, check ---\n"

        
    rntdic = {}    
    rntdic['tunehw'] = sorted(set(hwreturn))
    rntdic['hwofDeriv'] = sorted(set(hwofDeriv))
    rntdic['tuneDeriv'] = sorted(set(fineDeriv))
    rntdic['hwofInfl'] = sorted(set(hwofInfl))
    rntdic['tuneInfl'] = sorted(set(fineInfl))
    rntdic['rawhwderiv'] = sorted(set(wordhw + wordhwL + wordhwC + wordDeriv + wordInfl))
    rntdic['notinben'] = sorted(set(wordnotBen))
    
    return rntdic
    


def tuneReport(wlist=[], path='', wfile=''):
    if wlist: fname = 'tuneReport-list.txt'
    elif wfile: fname = 'tuneReport-' + wfile
    else:
        print 'arguments error'
        return -1
    
    rnt = tuneBymwBen(wlist, path, wfile)
    
    tunehw = rnt['tunehw']
    hwofDeriv = rnt['hwofDeriv']
    tuneDeriv = rnt['tuneDeriv'] 
    hwofInfl = rnt['hwofInfl']
    tuneInfl = rnt['tuneInfl']
    notinben = rnt['notinben']    
    #rawhwderiv = rnt['rawhwderiv']
    
    fd = codecs.open('E:\\1Now\\mae\\report\\' + fname, 'w', 'utf-8')
    if notinben:
        fd.write('---not in mwBen---:' + str(len(notinben)) + '\n')
        for x in notinben: fd.write(x + '\n')
    if tunehw:
        fd.write('\n\n--- tuned hws of raw hws which are in mwBen---: ' + str(len(tunehw)) + '\n')
        for x in tunehw: fd.write(x + '\n')
    if hwofDeriv:
        fd.write('\n\n--- tuned hws of raw derivatives which are in mwBen---: ' + str(len(hwofDeriv)) + '\n')
        for x in hwofDeriv: fd.write(x + '\n')
    if tuneDeriv:
        fd.write('\n\n--- all tuned derivatives which are in mwBen---: ' + str(len(tuneDeriv)) + '\n')
        for x in tuneDeriv: fd.write(x + '\n') 
    if hwofInfl:
        fd.write('\n\n--- tuned hws of raw inflections which are in mwBen---: ' + str(len(hwofInfl)) + '\n')
        for x in hwofInfl: fd.write(x + '\n')
    if tuneInfl:
        fd.write('\n\n--- all tuned inflections which are in mwBen---: ' + str(len(tuneInfl)) + '\n')
        for x in tuneInfl: fd.write(x + '\n')             

    fd.close()
    return rnt
       

def txt2wl(path='', wfile=''):
    '''extract a word list from a text, return a sorted unqiue raw word list'''
    if wfile:
        with codecs.open(path + wfile, 'r', 'utf-8') as f:
            parags = f.read().splitlines() # each paragraph as one line
    else:
        print 'argument error'
        return -1
    
    wlraw = [] # raw word list from the text
    for parag in parags:
        regxp = re.compile(r'[^\w\'\-]+', re.U) # regular expression pattern: ^\w\'\- means not words ' -
        parag = regxp.sub(' ', parag) # replace all not words ' - with a space
        wlraw += parag.split(' ')
    
    return sorted(set(wlraw))



def hw4learn(wdict={}, wlist=[], path='', wfile='', excluf='E:\\1Now\\taglist\\2939exclu.txt'):
    '''This function only excludes hws and their derivatives exactly in mwl.
    So it's better to use tuneBymwBen() first to get a fine word list'''
    
    dictflag = 0 # no dictionary input
    
    if wdict:
        dictflag = 1
        words = wdict['tunehw'] + wdict['hwofDeriv'] + wdict['hwofInfl']
        sounds = wdict['tunehw'] + wdict['hwofInfl'] + wdict['tuneDeriv']
    elif wlist:
        words = [x.strip() for x in wlist]
    elif wfile:
        with codecs.open(path + wfile, 'r', 'utf-8') as f:
            words = f.read().splitlines()
        words = [x.strip() for x in words]    
    else:
        print 'argument error'
        return -1
    
    
    word4learn = []    
    if excluf:
        #print '--- will exclude words from', excluf
        with codecs.open(excluf, 'r', 'utf-8') as f:
            bexclus = f.read().splitlines()
    
        for word in words:
            if word not in bexclus: word4learn.append(word)
    else:
        word4learn = words
            
    word4learn = sorted(set(word4learn))
    
    # generate Anki search string for filtered deck    
    ankiSearchStr = 'deck:sPlay ('
    for x in word4learn:
        if x != word4learn[-1]: ankiSearchStr += ' hw:"' + x + '" or'
        else: ankiSearchStr += ' hw:"' + x + '")'
    print '\n---words to learn---: ', len(word4learn)
    print ankiSearchStr
    
    
    if dictflag:
        sound4learn = []
        if excluf:
            for x in sounds:
                if x not in bexclus: sound4learn.append(x)
        else:
            sound4learn = sounds
        sound4learn = sorted(set(sound4learn))
    
        ankiSearchStr = 'deck:soundPlay ('
        for x in sound4learn:
            if x != sound4learn[-1]: ankiSearchStr += ' hw:"' + x + '" or'
            else: ankiSearchStr += ' hw:"' + x + '")'
        print '\n---sounds to learn---: ', len(sound4learn)
        print ankiSearchStr    
        
        return sound4learn
        
#     return word4learn


def compareWordlist(l1=[], l2=[], path='', f1='', f2=''):
    '''get common and different parts of two lists or word lists of two files'''
    
    if l1:
        words1 = l1
#         words1 = [x.strip() for x in l1]
        f1name = 'list-1'
    if l2:
        words2 = l2
#         words2 = [x.strip() for x in l2]
        f2name = 'list-2'
    if f1:
        with codecs.open(path+f1, 'r', 'utf-8') as f:
            words1 = f.read().splitlines()
#         words1 = [x.strip() for x in words1]
        f1name = f1
    if f2:
        with codecs.open(path+f2, 'r', 'utf-8') as f:
            words2 = f.read().splitlines()
#         words2 = [x.strip() for x in words2]
        f2name = f2
            
    comm = [x for x in words1 if x in words2]
    only1 = [x for x in words1 if x not in words2]
    only2 = [x for x in words2 if x not in words1]
    
    comm = sorted(set(comm))
    
    f = codecs.open('E:\\1Now\\taglist\\report\\compare_' + f1name + '_' + f2name + '.txt', 'w', 'utf-8')
    
    print '---common words of ' + f1name + ' and ' + f2name + '---: ' + str(len(comm))
    f.write('---common words of ' + f1name + ' and ' + f2name + '---: ' + str(len(comm)) + '\n')
    for x in comm:
        f.write(x + '\n')
    
    print '---words only in ' + f1name + '---: ' + str(len(only1))
    f.write('\n\n---words only in ' + f1name + '---: ' + str(len(only1)) + '\n')
    for x in only1:
        f.write(x + '\n')    
    
    print '---words only in ' + f2name + '---: ' + str(len(only2))
    f.write('\n\n---words only in ' + f2name + '---: ' + str(len(only2)) + '\n')
    for x in only2:
        f.write(x + '\n')
        
    f.close()

    
print '--- load 5 functions ---'    

--- load 5 functions ---


In [21]:
# check number of words and notes in a filtered deck
# first export the filted deck in Anki

import codecs
with codecs.open('E:\\1Now\\mae\\article\\filter_46sounds1.txt', 'r', 'utf-8') as f:
    lines = f.read().splitlines()
words = [x.split('\t')[0] for x in lines]
words = sorted(words)
print 'number of unique words : ', len(set(words))

b = {}
for x in words:     
    b[x] = words.count(x)
#     print x, b[x]
print len(b)

cnt = 0

for x in b:
    #print x, b[x]
    cnt += b[x]
print 'number of notes in Anki: ', cnt

number of unique words :  134
134
number of notes in Anki:  134


In [2]:
# Organize 'beautifulExcluHw.txt' and generate 'beautifulExclu.txt'
# (note: words in beautifulExcluHw.txt are absolutely hws in 39195 of mwl)
# directly from mwaled mongodb dpdict. words not being a hw in mwaled won't be added to the two lists

import pymongo
from pymongo import MongoClient

# codecs if for read and write files with unicode, encoding 'utf-8'
import codecs

#connect to MongoDB
client = MongoClient('localhost', 27017)
db = client.dpdb  # databas: dpdb
geweiDict = db.geweiDict201511 # collection (table): geweiDict201511
#geweiDict = db.geweiDictTest


with codecs.open("E:\\1Now\\taglist\\2939bw.txt", 'r', 'utf-8') as f:
    behws = f.read().splitlines()
    f.close()    
# sort and make unique words in beautifulExcluHw.txt
behws = sorted(set(behws))
print 'number of excluded hw: ', len(behws)

# fd = codecs.open("E:\\1Now\\mae\\beautifulExcluHw.txt", 'w', 'utf-8')
# for behw in behws:
#     fd.write(behw + '\n')
# fd.close()
    
fdict = codecs.open('E:\\1Now\\mae\\beautifulExcluraw.txt', 'w', 'utf-8')
    
results = geweiDict.find({}, {"_id":0, "hw":1, "infl":1, "subhw":1}) #.limit(1000)

# total number of head words
totalhw = 0
totalcnt = 0

for result in results:    
    word = result.get("hw")    

    if word and word in behws:
        #print word
#         fdict.write(word + ';' + word + '\n')
        fdict.write(word + '\n')
        totalhw += 1
    else:
#         print "no head word"
        continue

    # get items of result
    infls = result.get("infl")
    subhws = result.get("subhw")     
    
    if infls:
        for infl in infls:
#             fdict.write(infl + ';' + word + '\n')
            fdict.write(infl + '\n')
            totalcnt += 1   
            
    if subhws:
        for subhw in subhws:
            if subhw[0] != word[0]: continue # check if subhw is a real sub hw of word by comparing first letter
#             fdict.write(subhw + ';' + word  + '\n')
            fdict.write(subhw + '\n')
            totalcnt += 1        
    
fdict.close()
print "total head words: ", totalhw
print "total words: ", totalcnt + totalhw


# sort out by head words and make them unique
with codecs.open('E:\\1Now\\mae\\beautifulExcluraw.txt', 'r', 'utf-8') as f:
    lines = f.read().splitlines()
lines = sorted(set(lines))
print 'unique words:', len(lines)
#lines = sorted(lines)

fd = codecs.open('E:\\1Now\\mae\\beautifulExclu.txt', 'w', 'utf-8')
for line in lines:
    fd.write(line + '\n')
fd.close()


number of excluded hw:  2939
total head words:  5011
total words:  31600
unique words: 9833


In [17]:
# output mwlBenchmark.txt (mwBen) (no duplicate lines)
# 2015-11-28, 2015-11-30, 2015-12-01, 2015-12-08

# sort and make every line unique
# format: n;hw, derivative, or inflection;hw
# 1 - head word; 2 - derivative; 3 - inflection

# 1;truly;truly
# 2;blazingly;blazing
# 3;blazes;blaze
# 3;blazing;blaze

# By the lines above, I can know blaze and blazing are hws, the two lines below are not allowd 
# because hws have derivates or inflections:
# blaze;blaze
# blazing;blazing
# the two lines above are redundant.

# manually delete: asas

import pymongo
from pymongo import MongoClient

# codecs if for read and write files with unicode, encoding 'utf-8'
import codecs

#connect to MongoDB
client = MongoClient('localhost', 27017)
db = client.dpdb  # databas: dpdb
geweiDict = db.geweiDict201511 # collection (table): geweiDict201511
#geweiDict = db.geweiDictTest

fdict = codecs.open('E:\\1Now\\taglist\\mwlBenchmarkraw.txt', 'w', 'utf-8')
    
results = geweiDict.find({}, {"_id":0, "hw":1, "infl":1, "subhw":1}) #.limit(1000)
# sort method will use much more memory
#results = geweiDict.find({}, {"_id":0, "hw":1, "prn":1, "alles":1, "idpvs":1, "sublist":1}).sort("hw", 1).limit(1000)


hwcnt = 0 # number of head words (hws)
# hwcapcnt = 0 # number of hws capitalized
inflcnt = 0 # number of inflections
subhwcnt = 0 # number of subhws
subhw1cnt = 0 # number of subhws whose first letter is not same as hw
onlyhwcnt = 0

# subhw1hw = []

for result in results:
    inflflag = 0
    subhwflag = 0
    
    word = result.get("hw")    
    if word:
        #print word
#         fdict.write(word + ';' + word + '\n')
        hwcnt += 1      
    else:
        print "no head word"
        continue

    # get items of result
    subhws = result.get("subhw")
    infls = result.get("infl")         
    
    if subhws:
        for subhw in subhws:
            #if subhw[0] != word[0]: continue       
            if subhw.lower() == word.lower():
                #print 'subhw == word', subhw
                continue           
            if infls:
                if subhw in infls:
                    #print '----subhw in infls:', subhw
                    continue
                    
            if subhw[0] != word[0]:
                #print 'subhw is not like word: ', subhw, '...', word
                subhw1cnt += 1
                #subhw1hw.append(subhw+';'+word)
            fdict.write('2;' + subhw + ';' + word + '\n')
            subhwcnt += 1      
            subhwflag = 1    
    
    
    if infls:
        for infl in infls:
            if infl.lower() == word.lower():
                #print 'infl == word', infl
                continue
            fdict.write('3;' + infl + ';' + word + '\n')
            inflcnt += 1   
            inflflag = 1
            
            
    # hw;hw (only when hw has no inflections and no subhws)
    if inflflag == 0 and subhwflag == 0:
        fdict.write('1;' + word + ';' + word + '\n')
        onlyhwcnt += 1
    
fdict.close()

print '--- mwBen statistics ---'
print "total head words: ", hwcnt
# print 'total hws capitalized: ', hwcapcnt
print "total inflections: ", inflcnt
print 'total subhws: ', subhwcnt
print 'total subhws whose 1st letters are different from hws: ', subhw1cnt
print 'hw without any derivatives: ', onlyhwcnt


with codecs.open('E:\\1Now\\taglist\\mwlBenchmarkraw.txt', 'r', 'utf-8') as f:
    mwls = f.read().splitlines()
print '\n\nnumber of lines in mwBen before adding 84en_fr.txt: ', len(mwls)
mwls = sorted(set(mwls))
print 'number of unique lines in mwBen before adding 84en_fr.txt: ', len(mwls)

# read en_fr.txt
with codecs.open('E:\\1Now\\taglist\\84en_fr.txt', 'r', 'utf-8') as f:
    enfrs = f.read().splitlines()
print 'number of lines in 84en_fr.txt: ', len(enfrs)
enfrs = sorted(set(enfrs))
print 'number of unique lines in 84en_fr.txt: ', len(set(enfrs))

compareWordlist(l1=enfrs, l2=mwls)

# sort and make unique
lines = sorted(set(mwls + enfrs))
print 'number of lines in mwBen after adding 84en_fr.txt: ', len(lines)

fd = codecs.open('E:\\1Now\\taglist\\mwlBenchmark.txt', 'w', 'utf-8')
for line in lines:
    fd.write(line + '\n')
fd.close()


# subhw1hw = sorted(set(subhw1hw))
# fd = codecs.open('E:\\1Now\\taglist\\subhw1hw.txt', 'w', 'utf-8')
# for x in subhw1hw:
#     fd.write(x + '\n')
# fd.close()


--- mwBen statistics ---
total head words:  39195
total inflections:  64355
total subhws:  8493
total subhws whose 1st letters are different from hws:  158
hw without any derivatives:  11221


number of lines in mwBen before adding 84en_fr.txt:  84069
number of unique lines in mwBen before adding 84en_fr.txt:  53853
number of lines in 84en_fr.txt:  84
number of unique lines in 84en_fr.txt:  83
---common words of list-1 and list-2---: 15
---words only in list-1---: 68
---words only in list-2---: 53838
number of lines in mwBen after adding 84en_fr.txt:  53921


In [15]:
# explore mwBen

import codecs
import string

with codecs.open('E:\\1Now\\taglist\\mwlBenchmark.txt', 'r', 'utf-8') as f:
    lines = f.read().splitlines()

deriv = [] # derivative
benhw = [] # hw
deriv1hw = [] # derivative (inflection, subhw) first letter different from its hw
hwcap = [] # hw whose first letter is capitalized
for line in lines:
    x1 = line.split(';')[0]
    x2 = line.split(';')[1]
    deriv.append(x1)
    benhw.append(x2)
    if x1[0] != x2[0]: deriv1hw.append(line)
    if x2[0] in string.uppercase[0:26]: hwcap.append(x2)
derivS = sorted(deriv)
benhwSU = sorted(set(benhw)) # Sorted, Unique
hwcap = sorted(set(hwcap))

print '--- explore mwBen ---'
print 'number of derivative column: ', len(deriv)
print 'number of hw column: ', len(benhw)
print 'unique number of hw column: ', len(benhwSU)
print 'number of derivatives (inflections and subhws) whose 1st letter different from hws: ', len(deriv1hw)
print 'number of capitalized hws: ', len(hwcap)

fout = codecs.open('E:\\1Now\\taglist\\mwlStat.txt', 'w', 'utf-8')
fout.write('--- statistics ---\n')
fout.write('number of lines in mwBen: ' + str(len(deriv)) + '\n')
fout.write('unique number of hw: ' + str(len(benhwSU)) + '\n')
fout.write('number of derivatives (inflections and subhws) whose 1st letter different from hws: ' + str(len(deriv1hw)) + '\n')
fout.write('number of capitalized hws: ' + str(len(hwcap)) + '\n')

fout.write('Contents\n--------\n')
fout.write("Q: Are there duplicate derivatives who aren't hws in mwBen?\n")
fout.write("Q: List derivatives (inflections and subhws) whose first letter is different from the first letter of their hws\n")
fout.write("Capitalized hws\n\n")

# Q: Are there duplicate derivatives who aren't hws in mwBen?
dupderiv = [x for x in derivS if x not in benhwSU and derivS.count(x)>1]
fout.write("--- Duplicate derivatives not being hws: ---\n")
for x in dupderiv:
    fout.write(x + '\n')

# Q: List subhws whose first letter is different from the first letter of their hws
fout.write("\n--- derivatives (inflections and subhws) whose 1st letter different from hws: ---\n")
fout.write("derivatives (inflections and subhws);hw\n\n")
for x in subhw1hw:
    fout.write(x + '\n')

fout.write("\n--- Capitalized hws: ---\n")
for x in hwcap:
    fout.write(x + '\n')
           
fout.close()    

--- explore mwBen ---
number of derivative column:  53972
number of hw column:  53972
unique number of hw column:  34398
number of derivatives (inflections and subhws) whose 1st letter different from hws:  337
number of capitalized hws:  1487
